In [71]:
#have to install 1 time and then put the first line in a comment
!pip install turicreate
import turicreate as tc
import numpy as np
import pandas as pd
import os
import tensorflow as T
import keras
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape,  Flatten, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop, Adamax
from keras.regularizers import l2
from keras.layers import Multiply, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from time import time
import multiprocessing as mp
import sys
import math
import argparse
import matplotlib.pyplot as plt
from IPython.display import display, HTML


In [0]:
#del os.environ['LC_ALL']
!pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
RATING_DATA_FILE_TRAIN = 'u1.base'
RATING_DATA_FILE_TEST = 'u1.test'
RATING_DATA_FILE_USER = 'u.user'
RATING_DATA_FILE_ITEM = 'u.item'
RATING_DATA_FILE_OCCUPATION = 'u.occupation'
RATING_DATA_GEN_FILE = 'u_gen.data'
RATINGS_CSV_FILE_NORM = 'u_norm.data'
RATINGS_GEN_CSV_FILE = 'u_genr.data'
MODEL_WEIGHTS_FILE = 'u_emb_weights.h5'
MODEL_WEIGHTS_FILE_CORE = 'u_emb_weights'

In [0]:
rating_file_import_train = drive.CreateFile({'id':'1SClzmjAoOCDY5kYWGHaA27RCkzqupddU'})
rating_file_import_train.GetContentFile(RATING_DATA_FILE_TRAIN)
rating_file_import_test = drive.CreateFile({'id':'1ELSUkW4DWU7TEWSto6nDBTewWLwHVCh2'})
rating_file_import_test.GetContentFile(RATING_DATA_FILE_TEST)
rating_file_import_user = drive.CreateFile({'id':'1m2UzDHT1lsLNE8TB6r3c7IIk2-iqbYfi'})
rating_file_import_user.GetContentFile(RATING_DATA_FILE_USER)
rating_file_import_item = drive.CreateFile({'id':'1F4E6SnymhtDs1F0kx0hR8cvBy6BquRrb'})
rating_file_import_item.GetContentFile(RATING_DATA_FILE_ITEM)
rating_file_import_occupation = drive.CreateFile({'id':'1POeYU9Vq67BWOZ0zBxg1poUiKhGXtaEZ'})
rating_file_import_occupation.GetContentFile(RATING_DATA_FILE_OCCUPATION)


In [75]:
ratings = pd.read_csv(RATING_DATA_FILE_TRAIN, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print(str(len(ratings))+' ratings loaded')

80000 ratings loaded


In [76]:
test_ratings = pd.read_csv(RATING_DATA_FILE_TEST, 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
test_ratings['user_emb_id'] = test_ratings['userid'] - 1
test_ratings['movie_emb_id'] = test_ratings['movieid'] - 1
print(str(len(test_ratings))+' ratings loaded')

20000 ratings loaded


In [0]:
items = pd.read_csv(RATING_DATA_FILE_ITEM,sep='|', engine='python',encoding='latin-1',names=['movieid','movie title','release date','video release date','IMDb URL'
          ,'unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western'])

In [0]:
ratings = pd.merge(ratings,items)
test_ratings = pd.merge(test_ratings,items)

In [79]:
Users = ratings['user_emb_id'].values
Movies = ratings['movie_emb_id'].values
Ratings = ratings['rating'].values
Items_Geners = ratings[['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']]


sf1 = tc.SFrame({"user_id":Users,"item_id":Movies,"rating":Ratings})
m1 = tc.ranking_factorization_recommender.create(sf1, target='rating')

sf2 = tc.SFrame({"user_id":Users,"item_id":Movies,"rating":Ratings})
m2 = tc.item_similarity_recommender.create(sf2, target='rating',similarity_type='pearson')

sf3 = tc.SFrame({"item_id":Movies,"item_data":Items_Geners})
m3 = tc.recommender.item_content_recommender.create(item_data=sf3,observation_data=sf1,user_id="user_id",target="rating",item_id='item_id')



Preparing data set.

Data has 80000 observations with 943 users and 1650 items.

Data prepared in: 0.097276s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Data has 80000 observations with 943 users and 1650 items.

Data prepared in: 0.108356s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.023899s

In [0]:
test_Users = test_ratings['user_emb_id'].values
test_Movies = test_ratings['movie_emb_id'].values
test_Items_Geners = test_ratings[['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']]

sf = tc.SFrame({"user_id":test_Users,"item_id":test_Movies})


df = test_ratings[["userid","movieid","rating"]]
df['prediction_matrix_factorization'] = m1.predict(sf)



display(df)


In [88]:
df['diff'] = abs((df['rating']-df['prediction_matrix_factorization']))
MAE = sum(df['diff'].values) / df.shape[0]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


1.1066947648274967